# Code Similarity with Fine-tuning

## Dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint as pp
import random
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset, random_split
# Hugging Face Transformers (fro CodeBERT etc.)
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
# Libraries for logging
from tqdm.auto import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

Device: cpu


## Seeding

In [3]:
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)

set_seed(seed_value=42)

## Dataset Access

In [4]:
#paired_dataset_url = "https://drive.google.com/uc?export=download&id=1fT4aEyIRzlqDcMFyyKkxgYN3-TGB5qqu"  # 40k dataset
paired_dataset_url = "https://drive.google.com/uc?export=download&id=1pUErbyZw1fBC5gIe6KT7BWga7h6Bfr4l"  # 100k dataset (TODO: train on this)

## Dataset

In [5]:
COLUMNS = [
    'pid',    # CodeNet problem ID
    'sid_1',  # CodeNet solution ID of 'src_1'
    'sid_2',  # CodeNet solution ID of 'src_2'
    'src_1',  # CodeNet solution code of 'sid_1'
    'src_2',  # CodeNet solution code of 'sid_2'
    'label',  # Label indicating if 'src_1' and 'src_2' both solve 'pid'
]

In [12]:
def downsample_df(df: pd.DataFrame, samples_per_class, seed=42):
    pos_df = df[df['label'] == 1]
    neg_df = df[df['label'] == 0]
    pos_sampled = neg_df.sample(n=min(samples_per_class, len(pos_df)), random_state=seed)
    neg_sampled = pos_df.sample(n=min(samples_per_class, len(neg_df)), random_state=seed)
    # Combine the downsampled dataframes
    downsampled_df = pd.concat([pos_sampled, neg_sampled]).reset_index(drop=True)
    return downsampled_df

In [13]:
df = pd.read_csv(paired_dataset_url, header=0, names=COLUMNS)
df = df.drop(columns=['pid', 'sid_1', 'sid_2'])
print(df['label'].value_counts())
df.head()

label
1    50000
0    50000
Name: count, dtype: int64


,src_1,src_2,label
0,"import math\n\na, b, h, m = map(int, input().s...","import math\nA,B,H,M=map(int,input().split())\...",1
1,n = input()\nprint(n[0:3]),\ns = input()\nprint(s[:3]),1
2,"n=int(input())\na=sorted(list(map(int,input()....","n = int(input())\na = list(map(int, input().sp...",1
3,# -*- coding: utf-8 -*-\n\ns = sorted(list(map...,"a,b,c = sorted(map(int,input().split()))\n\npr...",1
4,"N, K = map(int,input().split())\nR,S,P = map(i...","n, k = map(int, input().split())\ns, p, r = ma...",1


In [14]:
# Code datasets (for labeled and unlabeled code snippets)

class CodePairDataset(Dataset):
    def __init__(self, codes_a, codes_b, labels, tokenizer):
        super().__init__()
        assert len(codes_a) == len(codes_b) == len(labels), "Length MUST match!"
        self.codes_a = codes_a
        self.codes_b = codes_b
        self.labels = labels
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        code_a = self.codes_a[idx]
        code_b = self.codes_b[idx]
        label = self.labels[idx]
        # Encode the sequences for sequence pair classification
        # ([CLS], code_a tokens , [SEP], code_b tokens, [SEP])
        encoding = self.tokenizer(
            code_a, code_b,
            padding='max_length',  # Pad to max_length
            max_length=self.tokenizer.model_max_length,
            truncation=True,       # Truncate to max_length
            return_tensors='pt'    # Return torch.Tensor objects
        )
        # Remove batch dimension
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        return encoding, label

    def __len__(self):
        return len(self.labels)

    @classmethod
    def from_pandas_df(cls, df: pd.DataFrame, tokenizer, num_pairs=5000):
        # Filter sequences that don't fit the model's max length
        def filter_too_long_sequences(row):
            source = row['src_1'] + row['src_2']
            tokens = tokenizer.encode(source, truncation=False)
            return len(tokens) <= tokenizer.model_max_length

        print('filtering dataset, this might take a while ...')
        df = df[df.apply(filter_too_long_sequences, axis=1)]
        print('filtered dataset:', df.shape)
        df = downsample_df(df, samples_per_class=(num_pairs // 2))
        print('downsampled dataset:', df.shape)

        codes_a = df['src_1'].to_list()
        codes_b = df['src_2'].to_list()
        labels = df['label'].to_list()

        return cls(codes_a, codes_b, labels, tokenizer)

## Model

In [ ]:
# Model for finetuning

BEST_MODEL_PATH = "best_model.pt"

class CodeSimilarityClassifier(nn.Module):
    def __init__(self,
        bert,  # BERT based model instance
        freeze_bert=False,
        dropout_rate=0.2,
    ):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(dropout_rate)
        self.cls = nn.Linear(self.bert.config.hidden_size, 1)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, bert_input) -> torch.Tensor:
        with torch.device(device):
            bert_output = self.bert(**bert_input)
            pooler_output = bert_output.pooler_output
            pooler_output = self.drop(pooler_output)
            logits = self.cls(pooler_output)
            return logits

## Training

In [ ]:
def train_one_epoch(
    model, device,
    loss_func,
    scaler,
    optimizer,
    scheduler,
    dataloader,
    iters_to_accumulate,
    # for logging
    print_every=10
):
    running_loss = 0.0
    model.train()

    num_iter = len(dataloader)
    for iter, (encoding, labels) in enumerate(tqdm(dataloader)):
        # Converting to cuda tensors
        for k, v in encoding.items(): encoding[k] = v.to(device)
        labels = labels.to(device)

        # Obtaining the logits from the model
        # Enables autocasting for the forward pass (model + loss)
        with autocast('cuda'):
            # Obtaining the logits from the model
            logits = model(encoding)
            # Computing loss
            loss = loss_func(logits.squeeze(-1), labels.float())
            # Normalize the loss because it is averaged
            loss = loss / iters_to_accumulate

        # Backpropagating the gradients
        # Scales loss. (calls backward() on scaled loss to create scaled gradients)
        scaler.scale(loss).backward()

        if (iter + 1) % iters_to_accumulate == 0:
            # Optimization step
            # scaler.step() first unscales the gradients of the optimizer's assigned params.
            # If these gradients do not contain infs or NaNs, opti.step() is then called,
            # otherwise, opti.step() is skipped.
            scaler.step(optimizer)
            # Updates the scale for next iteration.
            scaler.update()
            # Adjust the learning rate based on the number of iterations.
            scheduler.step()
            # Clear gradients
            optimizer.zero_grad()

        running_loss += loss.item()

        # Print training loss information
        if (iter + 1) % print_every == 0:
            print(
                f"Iteration {iter+1}/{num_iter} complete. " +
                f"Loss: {running_loss / print_every}"
            )
            running_loss = 0.0


def evaluate_loss(
    model, device,
    loss_func,
    dataloader
):
    model.eval()

    loss, count = 0,0
    with torch.no_grad():
        for _, (encoding, labels) in enumerate(tqdm(dataloader)):
            # Converting to cuda tensors
            for k, v in encoding.items(): encoding[k] = v.to(device)
            labels = labels.to(device)
            logits = model(encoding)
            loss += loss_func(logits.squeeze(-1), labels.float()).item()
            count += 1

    mean_loss = loss / count
    return mean_loss


def train_bert(
    model,
    loss_func,
    optimizer,
    scheduler,
    train_loader, valid_loader,
    epochs,
    iters_to_accumulate
):
    best_loss = np.Inf
    train_losses = []
    valid_losses = []

    scaler = GradScaler('cuda')

    for epoch in range(epochs):
        # Compute training loss
        train_loss = train_one_epoch(
            model, device, loss_func,
            scaler,  # grad scaler
            optimizer,
            scheduler,
            train_loader,
            iters_to_accumulate,
            # print the training loss 5 times per epoch
            print_every=len(train_loader) // 5
        )
        # Compute validation loss
        valid_loss = evaluate_loss(
            model, device, loss_func,
            valid_loader
        )
        print(f"Epoch {epoch+1} complete! Validation Loss: {valid_loss}")

        if valid_loss < best_loss:
            print(f"Best validation loss improved from {best_loss} to {valid_loss}")
            best_loss = valid_loss
            torch.save(model.state_dict(), BEST_MODEL_PATH)

        torch.cuda.empty_cache()

In [ ]:
# Parameters

bert_name = "huggingface/CodeBERTa-small-v1"

"""
other checkpoints:
- "microsoft/codebert-base"
- "neulab/codebert-python"
"""

epochs = 4
lr = 1e-5  # learning rate
wd = 1e-5  # weight decay
bs = 20    # batch size
# The gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate.
# If set to "1", you get the usual batch size
iters_to_accumulate = 2

In [ ]:
# Dataset

bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)

dataset = CodePairDataset.from_pandas_df(df, bert_tokenizer, num_pairs=50_000)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_data, valid_data = random_split(dataset, [train_size, valid_size])

SHUFFLE = True
train_loader = DataLoader(train_data, batch_size=bs, shuffle=SHUFFLE)
valid_loader = DataLoader(valid_data, batch_size=bs, shuffle=SHUFFLE)

In [ ]:
# Model instance

bert_model = AutoModel.from_pretrained(bert_name).to(device)

model = CodeSimilarityClassifier(bert_model).to(device)

In [ ]:
# Optimizer and Scheduler

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

# The number of steps for the warmup phase.
num_warmup_steps = 0
# The total number of training steps
num_training_steps = epochs * len(train_loader)
# Necessary to take into account Gradient accumulation
num_training_steps = num_training_steps // iters_to_accumulate

scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [ ]:
# Train the model

loss_func = nn.BCEWithLogitsLoss()

train_bert(
    model, loss_func,
    optimizer,
    scheduler,
    train_loader, valid_loader,
    epochs,
    iters_to_accumulate
)

## Evaluation

In [ ]:
from torch.utils.data import Subset
from sklearn.metrics import classification_report

In [ ]:
def evaluate_preds(
    model, device,
    dataloader,
    threshold = 0.5
):
    model.eval()

    true_labels = []
    predictions = []

    with torch.no_grad():
        for _, (encoding, labels) in enumerate(tqdm(dataloader)):
            # Converting to CUDA tensors
            for k, v in encoding.items(): encoding[k] = v.to(device)
            labels = labels.to(device)

            # Get model logits and compute loss
            logits = model(encoding)
            # Convert logits to predictions (assuming binary classification with sigmoid)
            preds = (torch.sigmoid(logits.squeeze(-1)) > threshold).int()

            # Store predictions and labels
            true_labels.extend(labels.cpu().tolist())
            predictions.extend(preds.cpu().tolist())

    torch.cuda.empty_cache()
    return true_labels, predictions

In [ ]:
# TODO: maybe use something other than CodeNet
eval_data = Subset(valid_data, list(range(len(valid_data))))

# Load the best weights (state that achieved the lowest validation loss)
model.load_state_dict(torch.load(BEST_MODEL_PATH))

ls, ps = evaluate_preds(
    model, device,
    DataLoader(eval_data, batch_size=2, shuffle=True),
    threshold=0.5
)

In [ ]:
report = classification_report(ls, ps)
print(report)